In [1]:
#Original code for this cell at https://github.com/theofil/dscout/blob/master/analysis/exampleAnalysis.ipynb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import subprocess
import bz2
%matplotlib inline

plt.rc('font', size=15)



# data files
path = './data/hiion/' #  The files in this path have been manually processed (removed empty files, and merged a run with two parts into one)
files = os.listdir(path)
files = [f for f in files if 'monitor' not in f]

# getrunNumber from string e.g., int('/data/hiion/scout_326676_000000.monitor.txt'.split('_')[1]
getRun = lambda x: int(x.split('_')[1])



# list to hold dataframes
dfs = []
runIndex= []

for file in files:
    filepath = path+file
    df = pd.read_csv(filepath)
    
  
    # add a column with the run number
    df['run'] = getRun(file) 
    dfs += [df]
    
    #make in index with the run numbers
    runIndex += [getRun(file)]
plt.rc('figure', max_open_warning = 100)




In [2]:
#Fix the runs where the orbit number resets (the warning does not matter)

for df in dfs:
    if not df['orbit'].is_monotonic:
        for i in range(1,len(df)):
            if df['orbit'][i-1]>df['orbit'][i]:
                tmp = i-1
                break
        df['orbit'][tmp+1:] = df['orbit'][tmp] + df['orbit'][tmp+1:]
            

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [3]:
badruns = [0, 1, 4, 8, 9, 11, 28, 30, 33, 34, 35, 36, 39, 46, 47, 50,  57, 65, 68, 69, 70, 72, 77, 78]
goodruns = [i for i in range(len(dfs)) if i not in badruns]

In [4]:
dfg = [] #data frame with only goodruns

for i in range(len(dfs)):
    if i in goodruns : dfg.append(dfs[i])

In [6]:
# Make every df start at 1
for df in dfg:
    df['orbit'] = df['orbit'] - df['orbit'][0] + 1

In [7]:
for j in range(1, len(dfg)):
    dfg[j]['orbit']+=dfg[j-1]['orbit'].tolist()[-1]

In [8]:
merged = pd.concat(dfg, ignore_index=True, sort=False)


In [9]:
merged.to_csv(path_or_buf="merged.txt", index=False ) 
# so that someone else can intstant access to the data without running this code